In [1]:
"""
    knot(npts, ord, centre=false, step=1.0)
Generate a `ord` order B-spline open uniform knot vector over `npts = n+1` control vertices.
An open uniform knot vector has multiplicity of knot values
 at the ends equals to the order `ord` of the B-Spline basis function.
Internal knot values are evenly spaced (uniform).
The resulting open uniform basis functions yield curves
 that behave most nearly like Bézier curves.
`centre` Boolean value indicates if the knots are centered in the
 zero value or if them goes from zero to `m-1` where `m=npts+ord`.
 If `true` is choosen, then `m` must be odd.
Necessary condition is that `npts>=ord` in order to correct build the knot vector.
---
# Arguments
- `npts::Int64`: the number of the `n+1` points of the controll polygon.
- `ord::Int64`: the order of the B-Spline (`degree k-1`).
- `centre::Bool=false`: if the knot is centered in zero.
- `step::Float64=1.0`: the step between the knots.
---
# Examples
```jldoctest
julia> knot(5, 2)
7-element Array{Float64,1}:
 0.0
 0.0
 1.0
 2.0
 3.0
 4.0
 4.0
```
```jldoctest
julia> knot(6, 3, false, 0.25)
9-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.25
 0.5
 0.75
 1.0
 1.0
 1.0
```
```jldoctest
julia> knot(5, 2, true)
7-element Array{Float64,1}:
 -2.0
 -2.0
 -1.0
 0.0
 1.0
 2.0
 2.0
```
---
_By Elia Onofri, Giuseppe Santorelli_
"""

function knot(npts::Int64, ord::Int64, centre::Bool=false, step::Float64=1.0)::Array{Float64}
    # Input Data Check
    @assert npts >= ord ("ERROR: ord > npts")
	
    local m::Int64 = npts+ord      		   # knot vector dimension
    local backstep::Float64 = 0.0   		   # backstep is the affine transition parameter for 0-central knots
    local x::Array{Float64} = Array{Float64}(m)    # knot vector output
	
    # Different preparation for centre knot vector
    if centre
        @assert m % 2 == 1 ("ERROR: no centring points available")
        backstep = step * (npts - ord + 1) / 2
    end
    
    x[1] = 0 - backstep
    for i = 2 : m
        if i > ord && i <= npts + 1
            x[i] = x[i-1] + step
        else
            x[i] = x[i-1]
        end
    end
    return x
end

knot

In [14]:
"""
UNTESTED::knotc(npts, ord, b)
Generate a nonuniform open knot vector proportional to the chord lengths between defining polygon vertices.
# Arguments
- `npts::Int64`: the number of the contol polygon vertices.
- `ord::Int64`: order of the basis function.
- `b::Array{Float64}`: array containing the contol polygon vertices.
---
# Examples
```jldoctest
julia> 
```
```jldoctest
julia> 
```
```jldoctest
julia> 
```
---
_Giuseppe Santorelli_
"""

function knotc(npts::Int64, ord::Int64, b::Array{Float64})::Array{Float64}
    
    @assert length(b) == 3 * npts ("ERROR: array b not matching with parameters")
    @assert npts >= ord ("ERROR: ord > npts")

    chord::Array{Float64} = Array{Float64}(31)
    m::Int64 = npts + ord
    n::Int64 = npts - 1
    x::Array{Float64} = Array{Float64}(m)
        
    maxchord = 0
    icount = 0
    
    # determine chord distance between defining polygon vertices and their sum
    for i = 4 : 3 : 3*npts
        icount = icount + 1
        xchord = b[i] - b[i - 3]
        ychord = b[i + 1] - b[i - 2]
        zchord = b[i + 2] - b[i - 1]
        chord[icount] = sqrt(xchord * xchord + ychord * ychord + zchord * zchord)
        maxchord = maxchord + chord[icount]
    end
    
    # multiplicity of c=order zeros at the beginning of the open knot vector
    for i = 1 : ord
        x[i] = 0
    end
    
    # generate the internal knot values
    for i = 1 : npts - ord
        csum = 0
        
        for j = 1 : i
            csum = csum + chord[j]
        end
        
        numerator = i / (npts - ord + 1) * chord[i + 1] + csum
        x[ord + i] = (numerator / maxchord) * (npts - ord + 1)
    end
    
    # multiplicity of c=order zeros at the end of the open knot vector
    for i = npts + 1 : m
        x[i] = npts - ord + 1
    end
    
    return x
    
end

knotc

In [7]:
"""
    knotu(npts, ord, step=1.0)
Generate a B-spline periodic uniform knot vector over `npts = n+1` control vertices.
In a periodic uniform knot vector the knot values are evenly spaced by `step`.
Moreover each base is a translate of the others as it could be defined like:
```math
    N_{i, k}(t) = N_{i-1, k}(t-1) = N_{i+1, k}(t+1)
```
---
# Arguments
- `npts::Int64`: the number of the `n+1` points of the controll polygon.
- `ord::Int64`: the order of the B-Spline (`deg = ord-1`).
- `step::Float64=1.0`: the step between the knots.
---
# Example
```jldoctest
julia> knotu(5, 2)
7-element Array{Float64,1}:
 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
```
```
julia> knotu(5, 4, 0.25)
9-element Array{Float64,1}:
 0.0 
 0.25
 0.5 
 0.75
 1.0 
 1.25
 1.5 
 1.75
 2.0
```
---
_By Elia Onofri, Giuseppe Santorelli_
"""

function knotu(npts::Int64, ord::Int64, step::Float64=1.0)::Array{Float64}
    # Input Data Check
    @assert npts >= ord ("ERROR: ord > npts")
	
    local m::Int64 = npts + ord      		   # knot vector dimension
    local x::Array{Float64} = Array{Float64}(m)    # knot vector output

    for i = 1 : m
        x[i] = (i-1) * step
    end

    return x
end

knotu

In [15]:
?knotc

search:



UNTESTED::knotc(npts, ord, b) Generate a nonuniform open knot vector proportional to the chord lengths between defining polygon vertices.

# Arguments

  * `npts::Int64`: the number of the contol polygon vertices.
  * `ord::Int64`: order of the basis function.
  * `b::Array{Float64}`: array containing the contol polygon vertices.

---

# Examples

```jldoctest
julia> 
```

```jldoctest
julia> 
```

```jldoctest
julia> 
```

---

_Giuseppe Santorelli_


In [22]:
[0.0 2; 0 2; 2 0; 2 0]

4×2 Array{Float64,2}:
 0.0  2.0
 0.0  2.0
 2.0  0.0
 2.0  0.0

In [23]:
"""
    basis(ord, t, npts, x[])
Generate a B-spline basis functions `N[]` for a given open knot vector `x[]`.
A B-Spline basis is a collection of functions of a parameter `t` wich form
 a basis for the vectorial space of functions. The conformation of this set
 higly depends on the choosing of the knots `x[]` the curve is bound to.
The basis is computed with _Cox-de Boor_ recursive function applied to the
 basis dependency tree in order to optimise the computation.
---
# Arguments
- `ord::Int64`: the order of the B-Spline (`deg = ord-1`).
- `t::Float64`: the parameter value of the parametric curve.
- `npts::Int64`: the number of the points of the controll polygon.
- `x::Array{Float64}`: the knot vector.
---
_By Elia Onofri, Giuseppe Santorelli_
"""

function basis(ord::Int64, t::Float64, npts::Int64, x::Array{Float64})::Array{Float64}
    local max_N = npts-1+ord        # Needs i+(ord-1)|i=npts = npts-1+ord trivial basis
    local m::Int64 = npts+ord	    # Dimension of the knot vector
    local ddep::Float64 = 0.0       # Direct Dependency partial sum
    local fdep::Float64 = 0.0       # Forward Dependency partial sum
    
    local N::Array{Float64} = Array{Float64}(max_N)   # Basis progressive vector
    
    # Local check of the knot vector correctness
    @assert length(x) == m ("ERROR: incompatibile knot vector with given parameters n+1 = $(npts), k = $(ord)")
    
    # Eval N_{i,1} for i = 1:max_B
    for i = 1 : max_N
        if (t >= x[i]) && (t < x[i+1])
            N[i] = 1
        else
            N[i] = 0
        end
    end
    
    # Eval higher basis N_{i,deg} for deg = 2:ord and i = 1:max_B-deg
    for deg = 2 : ord
        for i = 1 : m-deg
            # Eval of the direct dependency
            if N[i] == 0
                ddep = 0.0
            else
                ddep = ((t - x[i]) * N[i]) / (x[i+deg-1] - x[i])
            end
            # Eval of the forward dependency
            if N[i+1] == 0
                fdep = 0.0
            else
                fdep = ((x[i+deg] - t) * N[i+1]) / (x[i+deg] - x[i+1])
            end
            # Collection of the dependencies
            N[i] = ddep + fdep
        end
    end
    
    # Otherwise last point is zero
    if t == x[m]
        N[npts] = 1
    end
    
    return N[1:npts]
end

basis

In [43]:
db(t) = dbasis(3, t, 5, knot(5,3))

db (generic function with 1 method)

In [48]:
db(1.0)

([0.0, 0.5, 0.5, 0.0, 0.0], [NaN, Inf, 2.75025e247, Inf, NaN], [NaN, Inf, 1.1001e248, NaN, NaN])

In [35]:
B = [2.0 0.0; 2.0 2.0; 0.0 2.0]::Array{Float64, 2};
n = length(B[:,1])::Int64 -1;
k = 3::Int64;
kvec = knot(n+1, k)::Array{Float64};

In [42]:
"""
    dbasis(ord, t, npts, x[])
Generate a B-spline basis functions `N[]` and its derivatives `d1[], d2[]` for a given open knot vector `x[]`.
A B-Spline basis is a collection of functions of a parameter `t` wich form
 a basis for the vectorial space of functions. The conformation of this set
 higly depends on the choosing of the knots `x[]` the curve is bound to.
The basis is computed with _Cox-de Boor_ recursive function applied to the
 basis dependency tree in order to optimise the computation.
---
# Arguments
- `ord::Int64`: the order of the B-Spline (`deg = ord-1`).
- `t::Float64`: the parameter value of the parametric curve.
- `npts::Int64`: the number of the points of the controll polygon.
- `x::Array{Float64}`: the knot vector.
---
_By Elia Onofri, Giuseppe Santorelli_
"""

function dbasis(ord::Int64, t::Float64, npts::Int64, x::Array{Float64})::Tuple{Array{Float64}, Array{Float64}, Array{Float64}}
    local max_N::Int64 = npts-1+ord		# Needs i+(ord-1)|i=npts = npts-1+ord trivial basis
    local m::Int64 = npts+ord			# Dimension of the knot vector
    local ddep::Float64 = 0.0   		# Direct Dependency partial sum
    local fdep::Float64 = 0.0   		# Forward Dependency partial sum
    
    local N::Array{Float64} = Array{Float64}(max_N)   # Basis progressive vector
    local d1::Array{Float64} = Array{Float64}(max_N)  # First Derivative progressive vector
    local d2::Array{Float64} = Array{Float64}(max_N)  # Second Derivative progressive vector
    
    # Local check of the knot vector correctness
    @assert length(x) == m ("ERROR: incompatibile knot vector with given parameters n+1 = $(npts), k = $(ord)")
    
    # Eval N_{i,1} for i = 1:max_B
    for i = 1 : max_N
        if (t >= x[i]) && (t < x[i+1])
            N[i] = 1
        else
            N[i] = 0
        end
    end
    
    if t == x[m]    # last(x)
        N[npts] = 1
    end
    
    # Eval higher basis N_{i,deg} for deg = 2:ord and i = 1:max_B-deg
    for deg = 2 : ord
        for i = 1 : m-deg
            
            # ----Eval of the direct dependency----
            if N[i] == 0
                ddep = 0.0
                f1 = 0.0
            else
                ddep = ((t - x[i]) * N[i]) / (x[i+deg-1] - x[i])
                f1 = N[i] / (x[i+deg-1] - x[i])
            end
            
            # ----Eval of the forward dependency----
            if N[i+1] == 0
                fdep = 0.0
                f2 = 0.0
            else
                fdep = ((x[i+deg] - t) * N[i+1]) / (x[i+deg] - x[i+1])
                f2 = -N[i+1] / (x[i+deg] - x[i+1])
            end

            #----Eval of the direct first derivative dependency----
            if d1[i] != 0    # if the lower order basis function is zero skip the calculation
                f3 = ((t - x[i]) * d1[i]) / (x[i+deg-1] - x[i])
                s1 = (2 * d1[i]) / (x[i+deg-1] - x[i])
            else
                f3 = 0.0
                s1 = 0.0
            end
            
            #----Eval of the forward first derivative dependency----
            if d1[i+1] != 0    # if the lower order basis function is zero skip the calculation
                f4 = ((x[i+deg] - t) * d1[i+1]) / (x[i+deg] - x[i+1])
                s2 = ((-2) * d1[i+1]) / (x[i+deg] - x[i+1])
            else
                f4 = 0.0
                s2 = 0.0
            end
		
            #----Eval of the direct second derivative dependency----
            if d2[i] != 0    # if the lower order basis function is zero skip the calculation
                s3 = ((t - x[i]) * d2[i] ) / (x[i+deg-1]-x[i])
            else
                s3 = 0.0
            end
            
            #----Eval of the forward second derivative dependency----
            if d2[i+1] != 0    # if the lower order basis function is zero skip the calculation
                s4 = ((x[i+deg] - t) * d2[i+1]) / (x[i+deg] - x[i+1])
            else
                s4 = 0.0
            end
			
            # Collection of the dependencies
            N[i] = ddep + fdep
            d1[i] = f1 + f2 + f3 + f4
            d2[i] = s1 + s2 + s3 + s4
        end
    end
    
    return (N[1:npts], d1[1:npts], d2[1:npts])
end

dbasis